## Laboratorium 5 - algorytm Najbliższej Średniej (NM)


### Opis
Celem laboratorium jest implementacja klasyfikatora najbliższej średniej NM (*Nearest Mean*).


### Zadanie 1
* Wczytaj dane.
* Wszystkie poniższe zadania wykonaj dla wszystkich dostępnych klas i wszystkich cech.


In [1]:
import numpy as np

with open('./dataset/dataset.npz', 'rb') as f:
    data = np.load(f)
    train, test = data['train'], data['test']

features = list(range(2, 16))

X_train = train[:, features]
Y_train = train[:, 0]

X_test = test[:, features]
Y_test = test[:, 0]

print(f"X shapes {X_train.shape}, {X_test.shape}")
print(f"Y shapes {Y_train.shape}, {Y_test.shape}")

X shapes (2244, 14), (1496, 14)
Y shapes (2244,), (1496,)


### Zadanie 2
Zaimplementuj klasyfikator najbliższej średniej (NM) z zastosowaniem odległości Euklidesa i wykonaj klasyfikację. Wyświetl wynik klasyfikacji (accuracy).

In [2]:
from sklearn.metrics import accuracy_score

def calculate_class_means(X, Y):
    classes = np.unique(Y)
    means = {}
    for cls in classes:
        indices = np.where(Y == cls)[0]
        means[cls] = np.mean(X[indices], axis=0)
    return means

def nearest_mean_classifier(X_train, Y_train, X_test):
    means = calculate_class_means(X_train, Y_train)
    predictions = []
    for sample in X_test:
        distances = {cls: np.linalg.norm(sample - mean) for cls, mean in means.items()}
        predicted_class = min(distances, key=distances.get)
        predictions.append(predicted_class)
    return np.array(predictions)

predictions = nearest_mean_classifier(X_train, Y_train, X_test)

accuracy = accuracy_score(Y_test, predictions)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 46.06%


### Zadanie 3
Zaimplementuj funkcję, która zwraca macierz kowariancji (*uwaga: biblioteka `numpy` posiada gotową implementację `cov` z którą powinieneś porównać swój wynik*).

\begin{equation*}
C = \frac{1}{n - 1} (X - \bar X)(X - \bar X)^T
\end{equation*}

gdzie:
* $X$ to macierz danych,
* $\bar X$ to wektor średnich wartości cech. 



In [3]:
def calculate_covariance_matrix(X):
    n = X.shape[0]
    mean_vector = np.mean(X, axis=0)
    centered_X = X - mean_vector
    covariance_matrix = (1 / (n - 1)) * np.dot(centered_X.T, centered_X)
    return covariance_matrix

custom_cov_matrix = calculate_covariance_matrix(X_train)

numpy_cov_matrix = np.cov(X_train, rowvar=False)

are_equal = np.allclose(custom_cov_matrix, numpy_cov_matrix, atol=1e-8)
print("Czy macierze są równe?:", are_equal)

Czy macierze są równe?: True


### Zadanie 4
Zaimplementuj klasyfikator najbliższej średniej (NM) z zastosowaniem odległości Mahalanobisa i wykonaj klasyfikację. Wyświetl wynik klasyfikacji (accuracy).

\begin{equation*}
D_j = \sqrt{ (x - \mu_j)^T S_j^{-1}(x - \mu_j) },
\end{equation*}

gdzie:
* $D_j$ to odległość klasyfikowanej próbki od klasy $j$, 
* $\mu_j$ to wektor średnich wartości cech dla klasy $j$, 
* $S_j^{-1}$ to macierz odwrotna do macierzy kowariancji klasy $j$, 
* a $x$ to klasyfikowana próbka.

> Podpowiedź: Do obliczenia macierzy odwrotnej możesz użyć funkcji `linalg.inv` z biblioteki `numpy`.

> UWAGA: W niniejszym zadaniu możesz zastosować dowolną strukturę kodu (nie musisz trzymać się struktury z poprzedniego zadania), jednak algorytm NM należy zaimplementować samodzielnie – bez użycia gotowych rozwiązań (np. z biblioteki `scikit-learn`).

<span style="text-decoration:underline">Referencje</span>

1. Mahalanobis, P C, _On test and measures of group divergence : theoretical formulae_, Journal and Proceedings of Asiatic Society of Bengal (New Series) Vol. 26, pp. 541-588. 1930. (URL: http://library.isical.ac.in:8080/xmlui/bitstream/handle/10263/1639/029.pdf)
2. McLachlan, Goeffrey J. _Mahalanobis distance_, Resonance, pp. 20-26. 1999. (URL: https://www.ias.ac.in/article/fulltext/reso/004/06/0020-0026)

In [4]:
def calculate_class_covariances(X, Y):
    classes = np.unique(Y)
    covariances = {}
    for cls in classes:
        indices = np.where(Y == cls)[0]
        X_class = X[indices]
        covariances[cls] = calculate_covariance_matrix(X_class)
    return covariances

def mahalanobis_distance(x, mean, cov_inv):
    diff = x - mean
    return np.sqrt(np.dot(np.dot(diff.T, cov_inv), diff))  

def nearest_mean_mahalanobis_classifier(X_train, Y_train, X_test):
    means = calculate_class_means(X_train, Y_train)
    covariances = calculate_class_covariances(X_train, Y_train)
    cov_inverses = {cls: np.linalg.inv(cov) for cls, cov in covariances.items()}
    predictions = []
    for sample in X_test:
        distances = {cls: mahalanobis_distance(sample, means[cls], cov_inverses[cls]) for cls in means}
        predicted_class = min(distances, key=distances.get)
        predictions.append(predicted_class)
    return np.array(predictions)

predictions_mahalanobis = nearest_mean_mahalanobis_classifier(X_train, Y_train, X_test)
accuracy_mahalanobis = accuracy_score(Y_test, predictions_mahalanobis)
print(f"Accuracy: {accuracy_mahalanobis * 100:.2f}%")


Accuracy: 56.35%
